**DEPRECATED: easy export to mobile failed**

In [20]:
!pip install -qq transformers

In [21]:
import os.path

import torch
import torch.nn as nn
from torch.utils.mobile_optimizer import optimize_for_mobile

from transformers import AutoFeatureExtractor, ASTForAudioClassification

In [22]:
extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
extractor

ASTFeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "ASTFeatureExtractor",
  "feature_size": 1,
  "max_length": 1024,
  "mean": -4.2677393,
  "num_mel_bins": 128,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000,
  "std": 4.5689974
}

In [23]:
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
model

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTAttention(
            (attention): ASTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
            (de

In [24]:
class ModelProbabilities(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return torch.sigmoid(self.model(input_values=x).logits)

model_prob = ModelProbabilities(model)

In [25]:
sampling_rate = 16000
sample_input = torch.distributions.uniform.Uniform(-10000, 10000).sample((sampling_rate * 10,))

In [29]:
sample_features = extractor(sample_input, sampling_rate, return_tensors="pt")['input_values']
sample_features.shape

torch.Size([1, 1024, 128])

In [30]:
model_prob(sample_features).shape

torch.Size([1, 527])

In [32]:
traced_model = torch.jit.trace(model_prob, sample_features)
traced_model_optimized = optimize_for_mobile(traced_model)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [35]:
def extractor_func(inputs):
    return extractor(inputs, sampling_rate, return_tensors="pt")['input_values']

traced_extractor = torch.jit.trace(extractor_func, sample_input)

traced_extractor_optimized = optimize_for_mobile(traced_extractor)

/usr/local/lib/python3.10/dist-packages/transformers/models/audio_spectrogram_transformer/feature_extraction_audio_spectrogram_transformer.py:118: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  waveform = torch.from_numpy(waveform).unsqueeze(0)
/usr/local/lib/python3.10/dist-packages/transformers/models/audio_spectrogram_transformer/feature_extraction_audio_spectrogram_transformer.py:148: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if difference > 0:
/usr/local/lib/python3.10/dist-packages/transformers/models/

TypeError: Got <class 'torch.jit.ScriptFunction'>, but ScriptModule is expected.

In [37]:
output_path = './drive/MyDrive/Projects/MiniSoundFinder_v2/exports/'

traced_model._save_for_lite_interpreter(
    os.path.join(output_path, 'ast.pt'))

traced_extractor._save_for_lite_interpreter(
    os.path.join(output_path, 'extractor.pt'))

AttributeError: 'torch._C.ScriptFunction' object has no attribute '_save_for_lite_interpreter'